<a href="https://colab.research.google.com/github/allenbiji/datascienceproject/blob/main/Recommendationsystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all necessary libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ipywidgets import *

In [3]:
from google.colab import files

# Upload files from your local machine
uploaded = files.upload()

# If you want to display the names of uploaded files
for filename in uploaded.keys():
    print(f'File uploaded: {filename}')


Saving movies.csv to movies.csv
File uploaded: movies.csv


In [4]:
movies=pd.read_csv('movies.csv')

In [5]:
from google.colab import files

# Upload files from your local machine
uploaded = files.upload()

# If you want to display the names of uploaded files
for filename in uploaded.keys():
    print(f'File uploaded: {filename}')

Saving ratings.csv to ratings.csv
File uploaded: ratings.csv


In [6]:
ratings=pd.read_csv('ratings.csv')

EDA


In [7]:
movies.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [9]:
movies.isna().sum()

,0
movieId,0
title,0
genres,0


In [10]:
ratings.isna().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0


Null values are not present


In [11]:
ratings.describe()

,userId,movieId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [12]:
movies['genres']=movies['genres'].str.split('|')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [13]:
movies2=movies.explode('genres')

In [14]:
movies2['genres'].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'War', 'Musical', 'Documentary',
       'Western', 'Film-Noir', '(no genres listed)'], dtype=object)

In [15]:
movies2=movies2[movies2['genres']!='(no genres listed)']

'No genres listed' is removed as it provides no meaningful value

In [16]:
movies2['genres'].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'War', 'Musical', 'Documentary',
       'Western', 'Film-Noir'], dtype=object)

In [17]:
merged_data=pd.merge(ratings,movies2,on='movieId',how='inner')
merged_data

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime
1,1,16,4.0,1217897793,Casino (1995),Drama
2,9,16,4.0,842686699,Casino (1995),Crime
3,9,16,4.0,842686699,Casino (1995),Drama
4,12,16,1.5,1144396284,Casino (1995),Crime
...,...,...,...,...,...,...
281892,668,140098,2.5,1450415424,Runoff (2015),Drama
281893,668,140816,2.5,1443288791,Tangerine (2015),Comedy
281894,668,140816,2.5,1443288791,Tangerine (2015),Drama
281895,668,142488,4.0,1451535844,Spotlight (2015),Thriller


In [18]:
merged_data.groupby('genres').agg({'title':'nunique','rating':['mean','size']}).rename(columns={'title':'Unique_movies','rating':'mean_rating'})

Unique_movies mean_rating       
                  nunique        mean   size
genres                                      
Action               1736    3.451450  31205
Adventure            1164    3.518027  23076
Animation             400    3.635350   5966
Children              540    3.439429   8098
Comedy               3513    3.420996  38055
Crime                1440    3.642392  18291
Documentary           415    3.643035   1206
Drama                5217    3.650266  46960
Fantasy               670    3.500459  10889
Film-Noir             195    3.913636   1210
Horror               1001    3.281097   7983
IMAX                  152    3.641821   3032
Musical               409    3.571962   4287
Mystery               675    3.652043   8320
Romance              1788    3.544255  19094
Sci-Fi                858    3.454481  16795
Thriller             2187    3.495561  29288
War                   503    3.783202   5828
Western               235    3.565687   2314

Popularity Recommender

In [19]:
popularity=merged_data.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()
popularity.columns=['Genres','Title','Average rating','Rating count']
popularity

,Genres,Title,Average rating,Rating count
0,Action,'71 (2014),3.500000,1
1,Action,'Hellboy': The Seeds of Creation (2004),3.000000,1
2,Action,10 to Midnight (1983),2.500000,1
3,Action,12 Rounds (2009),2.875000,4
4,Action,13 Assassins (Jûsan-nin no shikaku) (2010),3.500000,5
...,...,...,...,...
23093,Western,Wyatt Earp (1994),3.200000,30
23094,Western,Young Guns (1988),3.375000,36
23095,Western,Young Guns II (1990),3.083333,12
23096,Western,Young Ones (2014),2.000000,1


In [20]:
popularity[(popularity['Genres']=='Action')&(popularity['Rating count']>=50)].sort_values(by='Average rating',ascending=False).head(7)

,Genres,Title,Average rating,Rating count
1179,Action,Princess Mononoke (Mononoke-hime) (1997),4.384615,52
1076,Action,North by Northwest (1959),4.273973,73
975,Action,"Matrix, The (1999)",4.264368,261
1433,Action,Star Wars: Episode V - The Empire Strikes Back...,4.228070,228
1331,Action,Seven Samurai (Shichinin no samurai) (1954),4.217742,62
1199,Action,Raiders of the Lost Ark (Indiana Jones and the...,4.212054,224
747,Action,Inception (2010),4.189320,103


Here, we obtained the top 7 best rated movies in the action genre which had at least 50 ratings.
We convert this logic into a function.


In [21]:
def topnreccomendations(genre,threshold,topn):
   popularity=merged_data.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()
   popularity.columns=['Genres','Title','Average rating','Rating count']

   topmovies=popularity[(popularity['Genres']==genre)&(popularity['Rating count']>=threshold)].sort_values(by='Average rating',ascending=False).head(topn)

   topmovies['S.No.']=range(1,len(topmovies)+1)
   topmovies.index=range(0,len(topmovies))
   topmovies=topmovies[['S.No.','Title','Average rating','Rating count']]

   return topmovies[['S.No.','Title','Average rating','Rating count']]




In [22]:
topnreccomendations("Action",20,12)

,S.No.,Title,Average rating,Rating count
0,1,Princess Mononoke (Mononoke-hime) (1997),4.384615,52
1,2,North by Northwest (1959),4.273973,73
2,3,"Professional, The (Le professionnel) (1981)",4.272727,22
3,4,Henry V (1989),4.272727,22
4,5,"Matrix, The (1999)",4.264368,261
5,6,Once Upon a Time in the West (C'era una volta ...,4.260870,23
6,7,"Killer, The (Die xue shuang xiong) (1989)",4.250000,20
7,8,"Treasure of the Sierra Madre, The (1948)",4.250000,26
8,9,Star Wars: Episode V - The Empire Strikes Back...,4.228070,228
9,10,Sherlock Holmes: A Game of Shadows (2011),4.225000,20


Content based recommender



In [23]:
movies2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy


In [27]:
movies3=movies2.groupby('title').agg({'genres':lambda x: " ".join(x)}).reset_index()
movies3

,title,genres
0,'71 (2014),Action Drama Thriller War
1,'Hellboy': The Seeds of Creation (2004),Action Adventure Comedy Documentary Fantasy
2,'Round Midnight (1986),Drama Musical
3,'Til There Was You (1997),Drama Romance
4,"'burbs, The (1989)",Comedy
...,...,...
10315,loudQUIETloud: A Film About the Pixies (2006),Documentary
10316,xXx (2002),Action Crime Thriller
10317,xXx: State of the Union (2005),Action Crime Thriller
10318,¡Three Amigos! (1986),Comedy Western


In [28]:
tf=TfidfVectorizer(analyzer='word',ngram_range=(1,3),stop_words='english')
tf

TfidfVectorizer(ngram_range=(1, 3), stop_words='english')

In [29]:
tf_matrix=tf.fit_transform(movies3['genres'])

In [30]:
cosine_sim=cosine_similarity(tf_matrix,tf_matrix)

In [31]:
cosine_sim

array([[1.        , 0.02677945, 0.02931913, ..., 0.10229517, 0.        ,
        0.        ],
       [0.02677945, 1.        , 0.        , ..., 0.03626651, 0.02411583,
        0.02863994],
       [0.02931913, 0.        , 1.        , ..., 0.        , 0.        ,
        0.35526663],
       ...,
       [0.10229517, 0.03626651, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.02411583, 0.        , ..., 0.        , 1.        ,
        0.07090711],
       [0.        , 0.02863994, 0.35526663, ..., 0.        , 0.07090711,
        1.        ]])

In [32]:
movies3

,title,genres
0,'71 (2014),Action Drama Thriller War
1,'Hellboy': The Seeds of Creation (2004),Action Adventure Comedy Documentary Fantasy
2,'Round Midnight (1986),Drama Musical
3,'Til There Was You (1997),Drama Romance
4,"'burbs, The (1989)",Comedy
...,...,...
10315,loudQUIETloud: A Film About the Pixies (2006),Documentary
10316,xXx (2002),Action Crime Thriller
10317,xXx: State of the Union (2005),Action Crime Thriller
10318,¡Three Amigos! (1986),Comedy Western


In [34]:
indices=pd.Series(movies3.index,index=movies3['title'])
indices

,0
title,
'71 (2014),0
'Hellboy': The Seeds of Creation (2004),1
'Round Midnight (1986),2
'Til There Was You (1997),3
"'burbs, The (1989)",4
...,...
loudQUIETloud: A Film About the Pixies (2006),10315
xXx (2002),10316
xXx: State of the Union (2005),10317
